In [1]:
#!/usr/bin/env python
# coding: utf-8

import tensorflow.keras 
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, concatenate
from tensorflow.keras.layers import Conv3D, MaxPooling3D, BatchNormalization, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
import shutil as shutil
import os as os
import nibabel as nib
import tensorflow
import math

In [2]:
def curve(model, path):
    acc_fig_path = path + "/acc"
    loss_fig_path = path + "/loss"
    plt.plot(model.history['accuracy'])
    plt.plot(model.history['val_accuracy'])
    plt.title('model_accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig(acc_fig_path)
    plt.close()
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig(loss_fig_path)
    plt.close() 

In [3]:
def txt(model, path):
    acc_file_path = path + "/acc.txt"
    loss_file_path = path + "/loss.txt"
    acc_txt = open(acc_file_path,'w')
    loss_txt = open(loss_file_path,'w')
    
    acc = model.history['accuracy']
    val_acc = model.history['val_accuracy']
    loss = model.history['loss']
    val_loss = model.history['val_loss']
    for i in range (len(his.history['accuracy'])):
        acc_string = "epoch " + str(i+1) + ": acc = " + str(acc[i]) + "\t val_acc = " + str(val_acc[i]) + "\n"
        loss_string = "epoch " + str(i+1) + ": loss = " + str(loss[i]) + "\t val_loss = " + str(val_loss[i]) + "\n"
        acc_txt.write(acc_string)
        loss_txt.write(loss_string)
            

    acc_txt.close()
    loss_txt.close()

In [4]:
def txt_eva(string, path):
    eva_file_path = path + "/eva.txt"
    eva_txt = open(eva_file_path,'w')
    eva_txt.write(string)
    
    eva_txt.close()

In [5]:
def Alexnet5(width, height, depth, channel, num_classes):
    
    input_T1 = Input(shape=(width, height, depth, channel), name='input_T1')
    T1 = Conv3D(kernel_size=(5, 5, 5),filters=50,
                strides=(1,1,1))(input_T1)  
    T1 = Conv3D(kernel_size=(5, 5, 5),filters=50,
                strides=(2,2,2))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=100,padding = 'same',
                strides=(1,1,1))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=250,padding = 'same',
                strides=(2,2,2))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=400,padding = 'same',
                strides=(1,1,1))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=550,padding = 'same',
                strides=(2,2,2))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=700,padding = 'same',
                strides=(1,1,1))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=850,padding = 'same',
                strides=(2,2,2))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=1000,padding = 'same',
                strides=(1,1,1))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=1150,padding = 'same',
                strides=(2,2,2))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=1300,padding = 'same',
                strides=(1,1,1))(T1)
    T1 = Conv3D(kernel_size=(3, 3, 3),filters=1600,padding = 'same',
                strides=(2,2,2))(T1)
    
    x = Flatten()(T1)
     
#   要新增 activation 以及 fcn
    x = Activation('relu')(x)
    x = Dense(1000, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
              
    output = Dense(num_classes, activation='softmax', name='output')(x)

    model = Model(inputs=[input_T1], outputs=[output])
    optimizer = tensorflow.keras.optimizers.SGD(learning_rate=0.0001)

    model.compile(
        optimizer=optimizer,
        loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[tensorflow.keras.metrics.SparseCategoricalAccuracy(name="accuracy")],
    return model   


In [6]:
def nifti_file_read(path, target_size):
    try:
        data = nib.load(path)
        data = data.get_fdata()
        data = (data-data.min())/ (data.max()-data.min()) *255
        data = data[:,:,:,np.newaxis]
    except Exception as e:
        print(e)
    else:
        x = np.expand_dims(np.array(data), axis=0)
        return x       

In [7]:
def generator_four_img(path1, batch_size, class_array, target_size):
    
    # make data path list
    # make dir path list
    data_path_list = []
    for class_index in class_array :
        tmp_path = os.path.join(path1,class_index)
        data_path_list.append(tmp_path) 
        
    # make nii file path list
    nii_path = []
    for index in data_path_list:
        tmp_nii_path = os.listdir(index)
        for i in tmp_nii_path:
            tmp_path = os.path.join(index, i)
            nii_path.append(tmp_path)
            
#   make nii label
    nii_labels = []
    for path in nii_path:
        for index in range (len(class_array)):
            if ( "/" + class_array[index] + "/") in path:
                nii_labels.append(index)
                
    nii_labels_dict = {}
    for i in range (len(nii_path)):
        nii_labels_dict.setdefault(nii_path[i], nii_labels[i]) 
        
    steps = math.ceil(len(nii_path) / batch_size)    # 确定每轮有多少个batch
    print("Found %s images."%len(nii_path))
    while True:
        np.random.shuffle(nii_path)
        for i in range(steps):
            nii_batch_list = nii_path[i * batch_size : i * batch_size + batch_size]
            label_batch_list = []
            for label in nii_batch_list:
                label_batch_list.append(nii_labels_dict[label])
            
            x = [nifti_file_read(file, target_size) for file in nii_batch_list]
            batch_x = np.concatenate([array for array in x])
            
            # label 轉成 one hot
            y = [labels for labels in label_batch_list]
            y = np.expand_dims(np.array(y), axis=0)
            batch_y = np.concatenate([array for array in y]) 
            
            yield batch_x, batch_y


In [8]:
num_classes = 2
model = Alexnet5(181,217,181,1,num_classes)
model.summary()
class_array = ['AD', 'NC']


steps_per_epoch = 900
validation_steps = 100
test_steps = 66

num_epochs = 50
batch_size = 2

path1 = "/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/train"
path5 = "/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/valid"
path_test = "/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test" 

path = "/workspace/YL/ADNC/3D-CNN_paper/result_10fold/data_range_-40~85_random_arc1_SGD_fold4"

if not os.path.isdir(path):
    os.mkdir(path)
else:
    shutil.rmtree(path)
    os.mkdir(path)   

best_model_path = path + "/best_model_ADNC.h5"
model_path = path + "/model_ADNC.h5"

train_flow = generator_four_img(path1, batch_size, class_array, (181,217,181))
valid_flow = generator_four_img(path5, batch_size, class_array, (181,217,181))
test_flow = generator_four_img(path_test, batch_size, class_array, (181,217,181))

stop = tensorflow.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=8, verbose=1, mode='auto', restore_best_weights=False)
best_model = ModelCheckpoint(best_model_path, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
callbacks_list = [stop, best_model]

his = model.fit_generator(train_flow,
                steps_per_epoch=steps_per_epoch,
                epochs=num_epochs,
                verbose=1,
                callbacks=callbacks_list,
                validation_data=valid_flow,
                validation_steps=validation_steps)

curve(his, path)
txt(his, path)
model.save(model_path)

Model: "functional_1"

_________________________________________________________________

Layer (type)                 Output Shape              Param #   


input_T1 (InputLayer)        [(None, 181, 217, 181, 1) 0         

_________________________________________________________________

conv3d (Conv3D)              (None, 177, 213, 177, 50) 6300      

_________________________________________________________________

conv3d_1 (Conv3D)            (None, 87, 105, 87, 50)   312550    

_________________________________________________________________

conv3d_2 (Conv3D)            (None, 87, 105, 87, 100)  135100    

_________________________________________________________________

conv3d_3 (Conv3D)            (None, 44, 53, 44, 267)   721167    

_________________________________________________________________

conv3d_4 (Conv3D)            (None, 44, 53, 44, 434)   3129140   

_________________________________________________________________

conv3d_5 (Conv3D)            (None, 22

In [9]:
tensorflow.keras.models.load_model(model_path)
_, accuracy = model.evaluate(test_flow,
                             steps=test_steps,
                             verbose=1,)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

string = f"Test accuracy: {round(accuracy * 100, 2)}%" + "\n"
txt_eva(string, path)

Found 132 images.

66/66 [==============================] - 42s 632ms/step - loss: 0.1597 - accuracy: 0.9394

Test accuracy: 93.94%


In [10]:
from tensorflow.keras.models import load_model

import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import os

from scipy import ndimage

label = class_array
class_num = 2
test_path = []

original_path = path_test
result_path = path + "/result.txt"

for i in range (1):
    for j in range (class_num):
        walk_path = original_path + "/" + label[j]
        test_path.append(walk_path)

model = tensorflow.keras.models.load_model(model_path)
record = open(result_path,'w')

for i in range (class_num):
    
    AD = 0
    LMCI = 0
    EMCI = 0
    NC = 0

    result = ""
    
    
    if i == 0:
        result = result + "AD: \n"
    elif i ==1:
        result = result + "NC: \n"
    data_list1 = os.listdir(test_path[class_num*0+i]) 

    data_list1.sort(key=lambda x:str(x[:-7]))
    
    for j in range (len(data_list1)):
        test1 = test_path[class_num*0+i] + "/" + data_list1[j]
        print(test1)
        
        data = nifti_file_read(test1, (181,217,181))
        preds = model.predict([data])
        
        if preds.argmax() == 0:
            AD = AD + 1
            result = result + (data_list1[j][:-7]) + "\tAD" + "\n"
        elif preds.argmax() == 1:
            NC = NC + 1
            result = result + (data_list1[j][:-7]) + "\tNC" + "\n"

    result = result + "AD: " + str(AD) + " NC: " + str(NC) + "\n"
    record.write(result)
            
record.close()


/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w002_S_0816-MPRAGE-2006-09-29_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w003_S_4152-Sag_IR-SPGR-2011-08-30_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w003_S_5187-Sag_IR-SPGR-2013-09-13_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w005_S_5119-Sag_IR-FSPGR-2014-04-22_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w007_S_4568-Sag_IR-SPGR-2012-09-17_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w009_S_5037-MPRAGE-2014-02-11_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_paper/ADNI_T1_data_VBM_fill_with_0_range_-40~85_random_fold4/test/AD/w011_S_4912-MPRAGE-2012-11-16_AD.nii.gz

/workspace/YL/ADNC/3D-CNN_p